## Purpose: 

For leave one out experiments, context property vector feature, reducing the dimensionality of the properties using PCA

For data: Download from AWS S3 - table-linker-datasets/Experiments/context_vector_{train/dev}data into Experiments/ folders
Alternatively recreate the data by running the datasets with context_property_vector.py

Both forward and reverse properties are considered.


In [1]:

import pandas as pd
import glob
import os
import numpy as np
from ast import literal_eval
import scipy.sparse as sp
import pickle
from sklearn.decomposition import IncrementalPCA

In [2]:
all_train_files = '../Experiments/context_vector_train_data/'
all_dev_files = '../Experiments/context_vector_dev_data/'
all_properties = ''
all_inverse_properties = ''
N_COMPONENTS = 100

In [3]:

all_train_files_list = glob.glob(all_train_files + '/*/*.csv')
print(len(all_train_files_list))
all_dev_files_list = glob.glob(all_dev_files + '/*/*.csv')
print(len(all_dev_files_list))

properties_set = set()
inverse_properties = set()
for file in all_train_files_list:
    df = pd.read_csv(file)
    df['context_property_vector'] = df['context_property_vector'].apply(literal_eval)
    property_values = df['context_property_vector'].values
    for k in property_values:
        for prop in k:
            #print(prop)
            if '_' in prop:
                inverse_properties.add(prop)
            else:
                properties_set.add(prop)
for file in all_dev_files_list:
    df = pd.read_csv(file)
    df['context_property_vector'] = df['context_property_vector'].apply(literal_eval)
    property_values = df['context_property_vector'].values
    for k in property_values:
        for prop in k:
            #print(prop)
            if '_' in prop:
                inverse_properties.add(prop)
            else:
                properties_set.add(prop)    


748
211


/nas/home/hrathod/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (29,30,31,36,38,42,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/nas/home/hrathod/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (42,43,44,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/nas/home/hrathod/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/nas/home/hrathod/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (44,49) have mixed types.Specify dtype option on import or set low_mem

In [5]:
#all = properties_set.union(inverse_properties)
#len(all)

4546

In [8]:
#pickle.dump(all, open('all_properties.pkl', 'wb'))

In [3]:
all = pickle.load(open('all_properties.pkl', 'rb'))
len(all)

4546

In [4]:
###Importing all the libraries
import glob
import boto3
import time
import os
import pandas as pd
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
from argparse import ArgumentParser, Namespace
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from itertools import chain
import copy
import shutil
import pickle
from tqdm import tqdm

# Defining connection to s3

###Defining all the results path a
features = ["monge_elkan","monge_elkan_aliases","jaro_winkler",
            "levenshtein","singleton", "num_occurences"]


def read_file(key):
    #resp = s3.get_object(Bucket = bucket, Key = key)
    try:
        df = pd.read_csv(key, sep = ',')
    except pd.errors.EmptyDataError:
        df = ''
        print('Empty csv file!')
    return df

def save_file(key, content):
    s3_res = boto3.resource('s3')

    object = s3_res.Object(bucket, key)
    result = object.put(Body = content)

def merge_files(args):
    # datapath = args.train_path
    df_list = []
    for fn in args.train_files:
        fid = fn.split('/')[-1][:-4]
        dataset_id = fn.split('/')[-2]
        df = read_file(fn)
        if not isinstance(df, pd.DataFrame) :
            continue

        df['table_id'] = fid
        df['dataset_id'] = dataset_id
        df['context_score'].fillna(0.0, inplace=True)
        if 'column-id' not in df.columns:
            df['column-id'] = fn.split('/')[-1] + df['column'].astype('str')

        df = df[extra_feat]
        df_list.append(df)
    return pd.concat(df_list)


def compute_normalization_factor(args, all_data):
    min_max_scaler_path = args.min_max_scaler_path
    all_data_features = all_data[features]
    scaler = MinMaxScaler()
    scaler.fit(all_data_features)
    #pickle.dump(scaler, open('./tmp/min_max_scaler_path.pkl', 'wb'))
    #s3_1.Bucket('table-linker-datasets').upload_file('./tmp/min_max_scaler_path.pkl', min_max_scaler_path)

    #save_file(min_max_scaler_path, scaler)
    return scaler

def save_pickle(key, content):
    pass

# In[10]:


def generate_train_data(args, all_data, scaler, shuffle_by = None):
    num_cells_1 = 0
    scaler_path = args.min_max_scaler_path
    #scaler = pickle.load(open('./tmp/min_max_scaler_path.pkl', 'rb'))
    final_list = []
    sfeatures = copy.deepcopy(features) + ['context_property_vector']
    normalize_features = features
    evaluation_label = ['evaluation_label']
    positive_features_final = []
    negative_features_final = []
    super_groups = all_data.groupby(['column-id'])
    if shuffle_by == 'dataset':
        super_groups = all_data.groupby(['dataset_id'])
        for i, s_group in super_groups:
            pos_features_dataset = []
            neg_features_dataset = []
            grouped_obj = s_group.groupby(['column', 'row', 'column-id'])

            for cell in grouped_obj:
                num_cells_1 += 1
                cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
                pos_features = []
                neg_features = []
                a = cell[1][cell[1]['evaluation_label'] == 1]
                if a.empty:
                    continue
                pos_rows = cell[1][cell[1]['evaluation_label'].astype(int) == 1][sfeatures].to_numpy()
                if len(pos_rows) < 1:
                    continue
                if len(pos_rows) > 1:
                    print("here")
                for i in range(len(pos_rows)):
                    pos_features.append(pos_rows[i])
                neg_rows = cell[1][cell[1]['evaluation_label'].astype(int) == -1][sfeatures].to_numpy()
                for i in range(min(batch_size, len(neg_rows))):
                    neg_features.append(neg_rows[i])

                for k in range(len(neg_features) - len(pos_features)):
                    pos_features.append(random.choice(pos_rows))
                if len(pos_features) != len(neg_features):
                    continue
                pos_features_dataset.append(pos_features)
                neg_features_dataset.append(neg_features)
            if len(pos_features_dataset) > 0:
                c = list(zip(pos_features_dataset, neg_features_dataset))
                random.shuffle(c)
                pos_features_dataset, neg_features_dataset = zip(*c)
                positive_features_final.extend(pos_features_dataset)
                negative_features_final.extend(neg_features_dataset)
    elif shuffle_by == 'table':
        super_groups = all_data.groupby(['table_id'])
        for i, s_group in super_groups:
            pos_features_table = []
            neg_features_table = []
            file_name = i.split('-')[0]
            #print("entering ", file_name)
            grouped_obj = s_group.groupby(['column', 'row'])

            for cell in grouped_obj:
                num_cells_1 += 1
                cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
                pos_features = []
                neg_features = []
                a = cell[1][cell[1]['evaluation_label'] == 1]
                if a.empty:
                    continue
                pos_rows = cell[1][cell[1]['evaluation_label'].astype(int) == 1][sfeatures].to_numpy()
                if len(pos_rows) < 1:
                    continue
                if len(pos_rows) > 1:
                    print("here")
                for i in range(len(pos_rows)):
                    pos_features.append(pos_rows[i])
                neg_rows = cell[1][cell[1]['evaluation_label'].astype(int) == -1][sfeatures].to_numpy()
                for i in range(min(batch_size, len(neg_rows))):
                    neg_features.append(neg_rows[i])

                for k in range(len(neg_features) - len(pos_features)):
                    pos_features.append(random.choice(pos_rows))
                if len(pos_features) != len(neg_features):
                    continue
                random.shuffle(pos_features)

                random.shuffle(neg_features)
                pos_features_table.append(pos_features)
                neg_features_table.append(neg_features)
            if len(pos_features_table) > 0:
                c = list(zip(pos_features_table, neg_features_table))
                random.shuffle(c)
                pos_features_table, neg_features_table = zip(*c)
                positive_features_final.extend(pos_features_table)
                negative_features_final.extend(neg_features_table)
    else:
        for i, s_group in super_groups:
            file_name = i.split('-')[0]
            #print("entering ", file_name)
            grouped_obj = s_group.groupby(['column', 'row'])
            for cell in grouped_obj:
                num_cells_1 += 1
                cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
                pos_features = []
                neg_features = []
                a = cell[1][cell[1]['evaluation_label'] == 1]
                if a.empty:
                    continue
                pos_rows = cell[1][cell[1]['evaluation_label'].astype(int) == 1][sfeatures].to_numpy()
                if len(pos_rows) < 1:
                    continue
                if len(pos_rows) > 1:
                    print("here")
                for i in range(len(pos_rows)):
                    pos_features.append(pos_rows[i])
                neg_rows = cell[1][cell[1]['evaluation_label'].astype(int) == -1][sfeatures].to_numpy()
                for i in range(min(batch_size, len(neg_rows))):
                    neg_features.append(neg_rows[i])

                for k in range(len(neg_features) - len(pos_features)):
                    pos_features.append(random.choice(pos_rows))
                random.shuffle(pos_features)
                random.shuffle(neg_features)
                if len(pos_features) != len(neg_features):
                    print("HHHERRRERR")
                else:
                    positive_features_final.append(pos_features)
                    negative_features_final.append(neg_features)
                #print(len(positive_features_final), len(positive_features_final[3]))
                #print(len(negative_features_final), len(negative_features_final[3]))
    if shuffle_by == 'complete_shuffle':
        c = list(zip(positive_features_final, negative_features_final))
        random.shuffle(c)
        positive_features_final, positive_features_final = zip(*c)
    print(len(positive_features_final), len(positive_features_final[3]))
    print(len(negative_features_final), len(negative_features_final[3]))
    pickle.dump(positive_features_final, open('pos_semtab_all_feat_8.pkl', 'wb'))
    pickle.dump(negative_features_final, open('neg_semtab_all_feat_8.pkl', 'wb'))
    print(len(positive_features_final), len(positive_features_final[3]))
    print(len(negative_features_final), len(negative_features_final[3]))


def merge_eval_files(final_score_path):

    for fn in final_score_path:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = read_file(fn)
        if not isinstance(df, pd.DataFrame):
            continue
        df['table_id'] = fid
        df_list.append(df)
    return pd.concat(df_list)


def parse_eval_files_stats(eval_data, method):
    res = {}
    candidate_eval_data = eval_data.groupby(['table_id', 'column', 'row'])['table_id'].count().reset_index(name="count")
    res['num_tasks_with_gt'] = len(eval_data[pd.notna(eval_data['GT_kg_id'])].groupby(['table_id', 'column', 'row']))
    num_tasks_with_model_score_top_one_accurate = []
    num_tasks_with_model_score_top_five_accurate = []
    num_tasks_with_model_score_top_ten_accurate = []
    has_gt_list = []
    has_gt_in_candidate = []
    for i, row in candidate_eval_data.iterrows():
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[
            (eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) > 0
        if np.nan not in set(c_e_data['GT_kg_id']):
            has_gt_list.append(1)
        else:
            has_gt_list.append(0)
        if 1 in set(c_e_data['evaluation_label']):
            has_gt_in_candidate.append(1)
        else:
            has_gt_in_candidate.append(0)

        # rank on model score
        s_data = c_e_data.sort_values(by=[method], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_model_score_top_one_accurate.append(1)
        else:
            num_tasks_with_model_score_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_model_score_top_five_accurate.append(1)
        else:
            num_tasks_with_model_score_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_model_score_top_ten_accurate.append(1)
        else:
            num_tasks_with_model_score_top_ten_accurate.append(0)

    res['num_tasks_with_model_score_top_one_accurate'] = sum(num_tasks_with_model_score_top_one_accurate)
    res['num_tasks_with_model_score_top_five_accurate'] = sum(num_tasks_with_model_score_top_five_accurate)
    res['num_tasks_with_model_score_top_ten_accurate'] = sum(num_tasks_with_model_score_top_ten_accurate)
    return res, candidate_eval_data






In [5]:
def read_file(key):
    #resp = s3.get_object(Bucket = bucket, Key = key)
    try:
        df = pd.read_csv(key, sep = ',')
    except pd.errors.EmptyDataError:
        df = ''
        print('Empty csv file!')
    return df

In [6]:
train_files = []
for train_path in train_files_path:
    set_of_files = glob.glob(train_path + '/*.csv')
    train_files.extend(set_of_files)
len(train_files)

748

In [7]:
dev_files_path = ['../Experiments/context_vector_dev_data/' + i for i in experiment_train_data.split(',')]
dev_files = []
for dev_path in dev_files_path:
    set_of_files = glob.glob(dev_path + '/*.csv')
    dev_files.extend(set_of_files)
len(dev_files)

211

In [10]:
extra_feat = ['column-id', 'column', 'row', 'evaluation_label', 'dataset_id', 'table_id', 'context_property_vector']
for f in features:
    extra_feat.append(f)
print("------------------------------------")
gen_training_data_args = Namespace(train_files=train_files, pos_output=pos_output, neg_output=neg_output,min_max_scaler_path=min_max_scaler_path)
print("\n")
scaler = compute_normalization_factor(gen_training_data_args, all_data)
print("------------------------------------")


/nas/home/hrathod/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (29,30,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/nas/home/hrathod/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (29,30,31,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/nas/home/hrathod/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/nas/home/hrathod/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (29,30,31,36,38,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.use

In [ ]:
pickle.dump(scaler, open('min_max_scaler.pkl', 'wb'))


In [8]:
properties_list = list(all)
#col_indices_1 = {i:i for i in range(len(features))}
col_indices = {properties_list[i]:i for i in range(len(properties_list))}
def convert_to_matrix_vector(data: pd.DataFrame, properties_list:list):
    #col = properties_list

    col_used_up = set()
    col = list(range(0, len(properties_list)))
    row = list(range(0, len(data)))
    
    rows, cols, vals = [], [], []
    for rows_ind in range(len(data)):
         for cols_ind in range(len(data[rows_ind])):
                if isinstance(data[rows_ind][cols_ind], str):
                    props = literal_eval(data[rows_ind][cols_ind])

                    for prop in props:
                        rows.append(rows_ind)
                        cols.append(col_indices[prop])
                        col_used_up.add(cols_ind)
                        vals.append(props[prop])

    X = sp.csr_matrix((vals, (rows, cols)),  shape=(len(data), len(properties_list))).toarray()
    return X
    
        
    
    
        
    

In [17]:
batch_size = 8
gen_training_data_args = Namespace(train_files=train_files, pos_output=pos_output, neg_output=neg_output,min_max_scaler_path=min_max_scaler_path)
generate_train_data(gen_training_data_args, all_data, scaler, shuffle_by = 'table')

47325 8
47325 8
47325 8
47325 8


In [9]:
def generate_dataloader(positive_feat_path, negative_feat_path):
    pos_features = pickle.load(open(positive_feat_path, 'rb'))
    neg_features = pickle.load(open(negative_feat_path, 'rb'))
    print(pos_features[10])
    pos_features_flatten = list(chain.from_iterable(pos_features))
    neg_features_flatten = list(chain.from_iterable(neg_features))
    return pos_features_flatten, neg_features_flatten

In [10]:
pos, neg = generate_dataloader('pos_semtab_all_feat_8.pkl', 'neg_semtab_all_feat_8.pkl')

[array([1.0, 1.0, 1.0, 1.0, 1.0, 0.9463052250822914, 0.9999999999999992,
       0.6661640466737692, 1.0, '{}'], dtype=object), array([1.0, 1.0, 1.0, 1.0, 1.0, 0.9463052250822914, 0.9999999999999992,
       0.6661640466737692, 1.0, '{}'], dtype=object), array([1.0, 1.0, 1.0, 1.0, 1.0, 0.9463052250822914, 0.9999999999999992,
       0.6661640466737692, 1.0, '{}'], dtype=object), array([1.0, 1.0, 1.0, 1.0, 1.0, 0.9463052250822914, 0.9999999999999992,
       0.6661640466737692, 1.0, '{}'], dtype=object), array([1.0, 1.0, 1.0, 1.0, 1.0, 0.9463052250822914, 0.9999999999999992,
       0.6661640466737692, 1.0, '{}'], dtype=object), array([1.0, 1.0, 1.0, 1.0, 1.0, 0.9463052250822914, 0.9999999999999992,
       0.6661640466737692, 1.0, '{}'], dtype=object), array([1.0, 1.0, 1.0, 1.0, 1.0, 0.9463052250822914, 0.9999999999999992,
       0.6661640466737692, 1.0, '{}'], dtype=object), array([1.0, 1.0, 1.0, 1.0, 1.0, 0.9463052250822914, 0.9999999999999992,
       0.6661640466737692, 1.0, '{}'], dtype=

In [14]:
X_data = convert_to_matrix_vector(pos, list(all))
Y_data = convert_to_matrix_vector(neg, list(all))

In [15]:
len(X_data)

378557

In [13]:
val = np.array([[1]]*len(X_data))
X_data = np.concatenate((X_data, val), axis = 1)

KeyboardInterrupt: 

In [ ]:
val = np.array([[0]]*len(Y_data))
Y_data = np.concatenate((Y_data, val), axis = 1)

In [ ]:
# Fit with added parameter to recognize positive and negative
pca = IncrementalPCA(n_components = N_COMPONENTS)
pca.partial_fit(X_data)
pca.partial_fit(Y_data)

In [16]:
# No added parameter
res = np.concatenate((X_data, Y_data), axis=0)


In [21]:
pca = IncrementalPCA(n_components = N_COMPONENTS)
pca.fit(res)

IncrementalPCA(n_components=100)

In [22]:
pickle.dump(pca, open("pca_var_100.pkl", "wb"))